In [ ]:
%pip install matplotlib

In [ ]:
%pip install pandas

In [ ]:
%pip install opencv-python

In [ ]:
%pip install mediapipe

In [ ]:
pip install scikit-image

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import json
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import skimage
from skimage.transform import PiecewiseAffineTransform, warp

def imshow(img):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(img)
    return ax

uv_path = "/Users/enriko/Downloads/mediapipe-facemesh-main/data/uv_map.json" #ubah sesuai direktori uv_map.json personal di folder /data
uv_map_dict = json.load(open(uv_path))
uv_map = np.array([ (uv_map_dict["u"][str(i)],uv_map_dict["v"][str(i)]) for i in range(468)])

In [ ]:
def load_obj(obj_filename):
    with open(obj_filename, 'r') as f:
        lines = [line.strip() for line in f]

    verts, uvcoords = [], []
    faces, uv_faces = [], []
    if lines and isinstance(lines[0], bytes):
        lines = [el.decode("utf-8") for el in lines]

    for line in lines:
        tokens = line.strip().split()
        if line.startswith("v "):
            vert = [float(x) for x in tokens[1:4]]
            if len(vert) != 3:
                msg = "Vertex %s does not have 3 values. Line: %s"
                raise ValueError(msg % (str(vert), str(line)))
            verts.append(vert)
        elif line.startswith("vt "):
            tx = [float(x) for x in tokens[1:3]]
            if len(tx) != 2:
                raise ValueError(
                    "Texture %s does not have 2 values. Line: %s" % (str(tx), str(line))
                )
            uvcoords.append(tx)
        elif line.startswith("f "):
            face = tokens[1:]
            face_list = [f.split("/") for f in face]
            for vert_props in face_list:
                faces.append(int(vert_props[0]))
                if len(vert_props) > 1:
                    if vert_props[1] != "":
                        uv_faces.append(int(vert_props[1]))

    verts = np.array(verts)
    uvcoords = np.array(uvcoords)
    faces = np.array(faces); faces = faces.reshape(-1, 3) - 1
    uv_faces = np.array(uv_faces); uv_faces = uv_faces.reshape(-1, 3) - 1
    return (
        verts,
        uvcoords,
        faces,
        uv_faces
    )

def write_obj(obj_name,
              vertices,
              faces,
              texture_name = "texture.jpg",
              colors=None,
              texture=None,
              uvcoords=None,
              uvfaces=None,
              inverse_face_order=False,
              normal_map=None,
              ):

    if os.path.splitext(obj_name)[-1] != '.obj':
        obj_name = obj_name + '.obj'
    mtl_name = obj_name.replace('.obj', '.mtl')
    texture_name
    material_name = 'FaceTexture'

    faces = faces.copy()

    faces += 1
    if inverse_face_order:
        faces = faces[:, [2, 1, 0]]
        if uvfaces is not None:
            uvfaces = uvfaces[:, [2, 1, 0]]

    with open(obj_name, 'w') as f:
        if texture is not None:
            f.write('mtllib %s\n\n' % os.path.basename(mtl_name))

        if colors is None:
            for i in range(vertices.shape[0]):
                f.write('v {} {} {}\n'.format(vertices[i, 0], vertices[i, 1], vertices[i, 2]))
        else:
            for i in range(vertices.shape[0]):
                f.write('v {} {} {} {} {} {}\n'.format(vertices[i, 0], vertices[i, 1], vertices[i, 2], colors[i, 0], colors[i, 1], colors[i, 2]))

        if texture is None:
            for i in range(faces.shape[0]):
                f.write('f {} {} {}\n'.format(faces[i, 2], faces[i, 1], faces[i, 0]))
        else:
            for i in range(uvcoords.shape[0]):
                f.write('vt {} {}\n'.format(uvcoords[i,0], uvcoords[i,1]))
            f.write('usemtl %s\n' % material_name)
            uvfaces = uvfaces + 1
            for i in range(faces.shape[0]):
                f.write('f {}/{} {}/{} {}/{}\n'.format(
                    faces[i, 0], uvfaces[i, 0],
                    faces[i, 1], uvfaces[i, 1],
                    faces[i, 2], uvfaces[i, 2]
                )
                )
            with open(mtl_name, 'w') as f:
                f.write('newmtl %s\n' % material_name)
                s = 'map_Kd {}\n'.format(os.path.basename(texture_name))
                f.write(s)

                if normal_map is not None:
                    name, _ = os.path.splitext(obj_name)
                    normal_name = f'{name}_normals.png'
                    f.write(f'disp {normal_name}')

                    cv2.imwrite(
                        normal_name,
                        normal_map
                    )
            skimage.io.imsave(texture_name, texture)

In [ ]:
def process_image(face_mesh, image_file):
    img_ori = cv2.imread(image_file)
    H, W, _ = img_ori.shape
    results = face_mesh.process(cv2.cvtColor(img_ori, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks:
        face_landmarks = results.multi_face_landmarks[0]
        keypoints = [(W * point.x, H * point.y) for point in face_landmarks.landmark[:468]]
        formatted_keypoints = [f"{point[0]:.2f}, {point[1]:.2f}" for point in keypoints]
        return formatted_keypoints
    return None

def load_images_from_folder(folder, extensions=("jpg", "jpeg", "png")):
    image_files = []
    for root, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(extensions):
                image_files.append(os.path.join(root, file))
    return image_files

image_directory = "/Users/enriko/Downloads/mediapipe-facemesh-main/img" #ubah sesuai direktori gambar
output_csv = "/Users/enriko/Downloads/mediapipe-facemesh-main/output/landmarks.csv" #ubah sesuai direktori output

image_files = load_images_from_folder(image_directory)

for image_file in image_files:
    print("Detected image:", image_file)

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

columns = [f"Landmark_{i}" for i in range(468)]
landmarks_df = pd.DataFrame(columns=columns)

for image_file in image_files:
    keypoints = process_image(face_mesh, image_file)
    if keypoints is not None:
        landmarks_df.loc[os.path.basename(image_file)] = keypoints
    else:
        print("No face detected in:", image_file)

landmarks_df.to_csv(output_csv, index=False, header=False)
print("Landmarks saved to:", output_csv)